The following code scores decently, but is not my best scoring notebook (nor is it the fastest), please see my other notebooks on my profile for additional ideas.

The current notebook can be improved to:
* Incorporate weather data.
* Reduce/prioritize features to avoid certain features being overwhelmingly powerful or having outsized interactions on the predictions. (Regularization and other strategies can also be considered.)
* Incorporate improved model tuning, the current notebook takes a very long time to run and although the results are good, some of my other notebooks run faster with better results. The hope was that with a larger number of features and increased training that the notebook could reach a level of grokking but that did not occur. **Highly suggest pulling in weather data as well.**

In [1]:
################################################################################
# IMPORTS
################################################################################
# Standard libraries
import os
import warnings
warnings.filterwarnings('ignore')

# Data manipulation libraries
import numpy as np
import pandas as pd

# Machine learning libraries
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold

# Darts time series libraries
from darts import TimeSeries
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, InvertibleMapper, StaticCovariatesTransformer
from darts.dataprocessing.transformers.missing_values_filler import MissingValuesFiller
from darts.metrics import rmsle
from darts.models import LinearRegressionModel, LightGBMModel, XGBModel, CatBoostModel
from darts.models.filtering.moving_average_filter import MovingAverageFilter

# Progress bar
from tqdm.notebook import tqdm_notebook

################################################################################
# UTILITY FUNCTIONS
################################################################################
def cprint(title, *args):
    """
    Helper function to print messages with formatting
    """
    print(
        "="*len(title), title, "="*len(title),
        *args,
        sep="\n",
    )

################################################################################
# DATA LOADING
################################################################################
PATH = "data"

# Load main datasets
train = pd.read_csv(os.path.join(PATH, "train.csv"), parse_dates=["date"])
test = pd.read_csv(os.path.join(PATH, "test.csv"), parse_dates=["date"])

# Load supplementary datasets
oil = pd.read_csv(os.path.join(PATH, "oil.csv"), parse_dates=["date"]).rename(columns={"dcoilwtico": "oil"})
store = pd.read_csv(os.path.join(PATH, "stores.csv"))
transaction = pd.read_csv(os.path.join(PATH, "transactions.csv"), parse_dates=["date"])
holiday = pd.read_csv(os.path.join(PATH, "holidays_events.csv"), parse_dates=["date"])

################################################################################
# DATA EXPLORATION
################################################################################
# Extract basic information about the data
num_family = train.family.nunique()
num_store = train.store_nbr.nunique()
num_ts = train.groupby(["store_nbr", "family"]).ngroups
train_start = train.date.min().date()
train_end = train.date.max().date()
num_train_date = train.date.nunique()
train_len = (train_end - train_start).days + 1
test_start = test.date.min().date()
test_end = test.date.max().date()
num_test_date = test.date.nunique()
test_len = (test_end - test_start).days + 1

# Print basic information
cprint(
    "Basic information of data",
    f"Number of family types      : {num_family}",
    f"Number of stores            : {num_store}",
    f"Number of store-family pairs: {num_family * num_store}",
    f"Number of target series     : {num_ts}",
    "",
    f"Number of unique train dates: {num_train_date}",
    f"Train date range            : {train_len} days from {train_start} to {train_end}",
    f"Number of unique test dates : {num_test_date}",
    f"Test date range             : {test_len} days from {test_start} to {test_end}",
)

# Analyze missing dates in training data
missing_dates = pd.date_range(train_start, train_end).difference(train.date.unique())
missing_dates = missing_dates.strftime("%Y-%m-%d").tolist()

unique_dp_count = train.groupby(["store_nbr", "family"]).date.count().unique().tolist()

cprint(
    "Missing gaps in time series",
    f"List incl. unique counts of data points: {unique_dp_count}",
    f"Missing dates                          : {missing_dates}",
)

################################################################################
# DATA PREPROCESSING - TRAINING DATA
################################################################################
# Reindex training data to ensure complete date coverage
multi_idx = pd.MultiIndex.from_product(
    [pd.date_range(train_start, train_end), train.store_nbr.unique(), train.family.unique()],
    names=["date", "store_nbr", "family"],
)
train = train.set_index(["date", "store_nbr", "family"]).reindex(multi_idx).reset_index()

# Fill missing values in training data
train[["sales", "onpromotion"]] = train[["sales", "onpromotion"]].fillna(0.)
train.id = train.id.interpolate(method="linear")  # interpolate linearly as a filler for the 'id'

################################################################################
# DATA PREPROCESSING - OIL DATA
################################################################################
# Analyze missing oil data
missing_oil_dates = pd.date_range(train_start, test_end).difference(oil.date)
num_missing_oil_dates = len(missing_oil_dates)
num_wknd_missing = (missing_oil_dates.weekday >= 5).sum()
total_num_wknd = (pd.date_range(train_start, test_end).weekday >= 5).sum()

cprint(
    "Missing oil dates",
    f"Number of missing oil dates: {num_missing_oil_dates}",
    f"Number of weekends missing : {num_wknd_missing}",
    f"Total number of weekends   : {total_num_wknd}",
)

# Reindex oil data to ensure complete date coverage
oil = oil.merge(
    pd.DataFrame({"date": pd.date_range(train_start, test_end)}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

# Fill missing oil values using linear interpolation
oil.oil = oil.oil.interpolate(method="linear", limit_direction="both")

################################################################################
# DATA PREPROCESSING - TRANSACTION DATA
################################################################################
# Analyze missing transaction records
num_zero_sales = (train.groupby(["date", "store_nbr"]).sales.sum().eq(0)).sum()
total_rec = num_store * train_len
curr_rec = len(transaction.index)
missing_rec = total_rec - curr_rec - num_zero_sales

cprint(
    "Missing transaction records",
    f"Correct number of records: {total_rec}",
    "",
    f"Breakdown...",
    f"Current number of records: {curr_rec}",
    f"Number of zero sales     : {num_zero_sales}",
    f"Number of missing records: {missing_rec}",
)

# Compute total sales for each store
store_sales = train.groupby(["date", "store_nbr"]).sales.sum().reset_index()

# Reindex transaction data to ensure complete coverage
transaction = transaction.merge(
    store_sales,
    on=["date", "store_nbr"],
    how="outer",
).sort_values(["date", "store_nbr"], ignore_index=True)

# Fill missing transaction values
# Set transactions to 0 for days with zero sales
transaction.loc[transaction.sales.eq(0), "transactions"] = 0.
transaction = transaction.drop(columns=["sales"])

# Fill remaining missing values using linear interpolation per store
transaction.transactions = transaction.groupby("store_nbr", group_keys=False).transactions.apply(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

################################################################################
# DATA PREPROCESSING - HOLIDAY DATA - EXPLORATION
################################################################################
# Explore holiday categories
national_locale_name = sorted(holiday[holiday.locale.eq("National")].locale_name.unique().tolist())
regional_locale_name = sorted(holiday[holiday.locale.eq("Regional")].locale_name.unique().tolist())
local_locale_name = sorted(holiday[holiday.locale.eq("Local")].locale_name.unique().tolist())

cprint(
    "List of locale names for each holiday",
    "Locale names for national holidays:",
    national_locale_name,
    "",
    "Locale names for regional holidays:",
    regional_locale_name,
    "",
    "Locale names for local holidays:",
    local_locale_name,
)

################################################################################
# DATA PREPROCESSING - HOLIDAY DATA - CLEANING
################################################################################
# Helper function to process holiday descriptions
def process_holiday(s):
    """Clean and standardize holiday descriptions"""
    if "futbol" in s:
        return "futbol"
    to_remove = list(set(store.city.str.lower()) | set(store.state.str.lower()))
    for w in to_remove:
        s = s.replace(w, "")
    return s

# Process holiday descriptions
holiday.description = holiday.apply(
    lambda x: x.description.lower().replace(x.locale_name.lower(), ""), 
    axis=1,
).apply(
    process_holiday
).replace(
    r"[+-]\d+|\b(de|del|traslado|recupero|puente|-)\b", "", regex=True,
).replace(
    r"\s+|-", " ", regex=True,
).str.strip()

# Remove transferred holidays
holiday = holiday[holiday.transferred.eq(False)]

################################################################################
# DATA PREPROCESSING - HOLIDAY DATA - CATEGORIZATION
################################################################################
# 1. Extract work days (Saturdays designated as work days)
work_days = holiday[holiday.type.eq("Work Day")]
work_days = work_days[["date", "type"]].rename(
    columns={"type": "work_day"}
).reset_index(drop=True)
work_days.work_day = work_days.work_day.notna().astype(int)

# Remove work days from holiday dataframe
holiday = holiday[holiday.type!="Work Day"].reset_index(drop=True)

# 2. Process local holidays (city level)
local_holidays = holiday[holiday.locale.eq("Local")]
local_holidays = local_holidays[["date", "locale_name", "description"]].rename(
    columns={"locale_name": "city"}
).reset_index(drop=True)
local_holidays = local_holidays[~local_holidays.duplicated()]
local_holidays = pd.get_dummies(local_holidays, columns=["description"], prefix="loc")

# 3. Process regional holidays (state level)
regional_holidays = holiday[holiday.locale.eq("Regional")]
regional_holidays = regional_holidays[["date", "locale_name", "description"]].rename(
    columns={"locale_name": "state", "description": "provincializacion"}
).reset_index(drop=True)
regional_holidays.provincializacion = regional_holidays.provincializacion.eq("provincializacion").astype(int)

# 4. Process national holidays
national_holidays = holiday[holiday.locale.eq("National")]
national_holidays = national_holidays[["date", "description"]].reset_index(drop=True)
national_holidays = national_holidays[~national_holidays.duplicated()]
national_holidays = pd.get_dummies(national_holidays, columns=["description"], prefix="nat")

# Combine national holidays that fall on the same day
national_holidays = national_holidays.groupby("date").sum().reset_index()

# Shorten column name for visualization purposes
if "nat_primer grito independencia" in national_holidays.columns:
    national_holidays = national_holidays.rename(columns={"nat_primer grito independencia": "nat_primer grito"})

################################################################################
# DATA PREPROCESSING - FEATURE SELECTION AND COMBINATION
################################################################################
# Select national holidays with larger impacts on sales based on correlation analysis
selected_holidays = [
    "nat_terremoto", "nat_navidad", "nat_dia la madre", "nat_dia trabajo",
    "nat_primer dia ano", "nat_futbol", "nat_dia difuntos",
]

# Filter available holidays
available_holidays = [col for col in selected_holidays if col in national_holidays.columns]
keep_national_holidays = national_holidays[["date"] + available_holidays]

# Combine all datasets into a single comprehensive dataframe
data = pd.concat(
    [train, test], axis=0, ignore_index=True,
).merge(
    transaction, on=["date", "store_nbr"], how="left",
).merge(
    oil, on="date", how="left",
).merge(
    store, on="store_nbr", how="left",
).merge(
    work_days, on="date", how="left",
).merge(
    keep_national_holidays, on="date", how="left",
).sort_values(["date", "store_nbr", "family"], ignore_index=True)

# Fill holiday columns with 0s to indicate absence of holidays/events
data[["work_day"] + available_holidays] = data[["work_day"] + available_holidays].fillna(0)

# Include date-related features as future covariates
data["day"] = data.date.dt.day
data["month"] = data.date.dt.month
data["year"] = data.date.dt.year
data["day_of_week"] = data.date.dt.dayofweek
data["day_of_year"] = data.date.dt.dayofyear
data["week_of_year"] = data.date.dt.isocalendar().week.astype(int)
data["date_index"] = data.date.factorize()[0]  # Note: sort by date before computing this

# Add cyclical encoding for day of week, month, and day of year to better capture periodicity
data["sin_day_of_week"] = np.sin(2 * np.pi * data.day_of_week / 7)
data["cos_day_of_week"] = np.cos(2 * np.pi * data.day_of_week / 7)
data["sin_month"] = np.sin(2 * np.pi * data.month / 12)
data["cos_month"] = np.cos(2 * np.pi * data.month / 12)
data["sin_day_of_year"] = np.sin(2 * np.pi * data.day_of_year / 365)
data["cos_day_of_year"] = np.cos(2 * np.pi * data.day_of_year / 365)

# Add pay period indicators (based on domain knowledge: public sector paid on 15th and last day of month)
data["is_payday"] = ((data.date.dt.day == 15) | 
                     (data.date.dt.is_month_end)).astype(int)

# Add indicators for days after payday (spending often increases in days following payment)
data["is_day_after_payday"] = ((data.date.dt.day == 16) | 
                              (data.date.dt.day == 1)).astype(int)

data["is_two_days_after_payday"] = ((data.date.dt.day == 17) | 
                                   (data.date.dt.day == 2)).astype(int)

# Add payday week indicator (entire week after payday may see elevated sales)
data["is_payday_week"] = ((data.date.dt.day >= 15) & (data.date.dt.day <= 21) | 
                          (data.date.dt.day >= 1) & (data.date.dt.day <= 7)).astype(int)

# Impute days with zero sales using linear interpolation later
zero_sales_dates = missing_dates + [f"{j}-01-01" for j in range(2013, 2018)]
data.loc[(data.date.isin(zero_sales_dates))&(data.sales.eq(0))&(data.onpromotion.eq(0)), ["sales", "onpromotion"]] = np.nan

# Add prefixes for clarity in feature naming
data.store_nbr = data.store_nbr.apply(lambda x: (f"store_nbr_{x}"))
data.cluster = data.cluster.apply(lambda x: (f"cluster_{x}"))
data.type = data.type.apply(lambda x: (f"type_{x}"))

# Add prefixes to ensure no duplicate values between 'city' and 'state'
data.city = data.city.apply(lambda x: (f"city_{x.lower()}"))
data.state = data.state.apply(lambda x: (f"state_{x.lower()}"))

################################################################################
# FEATURE SELECTION - IDENTIFY LOW VARIANCE FEATURES
################################################################################
# Function to identify low variance features
def identify_low_variance_features(df, threshold=0.01):
    """
    Identify features with low variance that might not be useful for prediction.
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame containing features to analyze
    threshold : float
        Variance threshold
        
    Returns:
    --------
    list
        List of feature names with variance below threshold
    """
    # Select numeric columns
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    
    # Apply variance threshold
    selector = VarianceThreshold(threshold)
    selector.fit(df[numeric_cols])
    
    # Identify low variance features
    low_var_indices = np.where(selector.variances_ <= threshold)[0]
    low_var_features = numeric_cols[low_var_indices].tolist()
    
    return low_var_features

# Identify low variance features in the dataset
# We'll focus on sales data for variance analysis since it's our target
sales_columns = ['sales', 'onpromotion', 'transactions', 'oil'] + \
                [col for col in data.columns if col.startswith(('is_', 'work_day', 'nat_'))]

# Create a sample for variance analysis (to save computation)
sample_data = data.sample(frac=0.1, random_state=42)
low_var_features = identify_low_variance_features(sample_data[sales_columns], threshold=0.01)

# Print low variance features that will be excluded
cprint(
    "Low variance features",
    f"Features with variance below threshold: {low_var_features}",
)

# Exclude low variance features from our feature sets
excluded_features = set(low_var_features)

################################################################################
# FEATURE ENGINEERING - PIPELINE CREATION
################################################################################

def get_pipeline(static_covs_transform=False, log_transform=False):
    """
    Create a pipeline for data transformation.
    
    Parameters:
    -----------
    static_covs_transform : bool
        Whether to transform static covariates using OneHotEncoder
    log_transform : bool
        Whether to apply log transformation to the target variable
        
    Returns:
    --------
    Pipeline
        Darts pipeline for data transformation
    """
    lst = []
    
    # Fill missing values
    filler = MissingValuesFiller(n_jobs=-1)
    lst.append(filler)
    
    # Specify transformation for static covariates
    if static_covs_transform:
        static_covs_transformer = StaticCovariatesTransformer(
            transformer_cat=OneHotEncoder(),
            n_jobs=-1,
        )
        lst.append(static_covs_transformer)

    # Perform log transformation on sales
    if log_transform:
        log_transformer = InvertibleMapper(
            fn=np.log1p,
            inverse_fn=np.expm1,
            n_jobs=-1,
        )
        lst.append(log_transformer)

    # Rescale time series
    scaler = Scaler()
    lst.append(scaler)

    # Chain all transformations
    pipeline = Pipeline(lst)
    return pipeline

################################################################################
# TIME SERIES EXTRACTION FUNCTIONS
################################################################################

def get_target_series(static_cols, log_transform=True):
    """
    Extract target time series from the data.
    
    Parameters:
    -----------
    static_cols : list
        List of static covariates to include
    log_transform : bool
        Whether to apply log transformation to the target variable
        
    Returns:
    --------
    target_dict : dict
        Dictionary of target time series by product family
    pipe_dict : dict
        Dictionary of pipelines by product family
    id_dict : dict
        Dictionary of series identifiers by product family
    """
    target_dict = {}
    pipe_dict = {}
    id_dict = {}

    for fam in tqdm_notebook(data.family.unique(), desc="Extracting target series"):
        # Filter data for each model
        df = data[(data.family.eq(fam)) & (data.date.le(train_end.strftime("%Y-%m-%d")))]
        
        # Initialize transformation pipeline for target series
        pipe = get_pipeline(True, log_transform=log_transform)
        
        # Extract target series together with static covariates
        target = TimeSeries.from_group_dataframe(
            df=df,
            time_col="date",
            value_cols="sales",
            group_cols="store_nbr",
            static_cols=static_cols,
        )

        # Record identity of each target series
        target_id = [{"store_nbr": t.static_covariates.store_nbr[0], "family": fam} 
                     for t in target]
        id_dict[fam] = target_id
        
        # Apply transformations
        target = pipe.fit_transform(target)
        target_dict[fam] = [t.astype(np.float32) for t in target]
        pipe_dict[fam] = pipe[2:]
        
    return target_dict, pipe_dict, id_dict

def get_covariates(
    past_cols,
    future_cols,
    past_ma_cols=None,
    future_ma_cols=None,
    past_window_sizes=[7, 28],
    future_window_sizes=[7, 28],
):
    """
    Extract past and future covariates from the data.
    
    Parameters:
    -----------
    past_cols : list
        List of past covariates to include
    future_cols : list
        List of future covariates to include
    past_ma_cols : list or None
        List of past covariates to compute moving average
    future_ma_cols : list or None
        List of future covariates to compute moving average
    past_window_sizes : list
        Window sizes for past moving averages
    future_window_sizes : list
        Window sizes for future moving averages
        
    Returns:
    --------
    past_dict : dict
        Dictionary of past covariates by product family
    future_dict : dict
        Dictionary of future covariates by product family
    """
    past_dict = {}
    future_dict = {}
    
    # Initialize transformation pipeline for covariates
    covs_pipe = get_pipeline()

    for fam in tqdm_notebook(data.family.unique(), desc="Extracting covariates"):
        # Filter data for each model
        df = data[data.family.eq(fam)]
        
        # Extract past covariates - Include the full date range (training + test)
        past_covs = TimeSeries.from_group_dataframe(
            df=df,  # No date filtering here - include all dates
            time_col="date",
            value_cols=past_cols,
            group_cols="store_nbr",
        )
        past_covs = [p.with_static_covariates(None) for p in past_covs]
        past_covs = covs_pipe.fit_transform(past_covs)
        
        # Add moving average features for past covariates if specified
        if past_ma_cols is not None and len(past_ma_cols) > 0:
            for size in past_window_sizes:
                ma_filter = MovingAverageFilter(window=size)
                past_ma_cols_filtered = [col for col in past_ma_cols if col not in excluded_features]
                if past_ma_cols_filtered:
                    old_names = [f"rolling_mean_{size}_{col}" for col in past_ma_cols_filtered]
                    new_names = [f"{col}_ma{size}" for col in past_ma_cols_filtered]
                    past_ma_covs = [
                        ma_filter.filter(p[past_ma_cols_filtered]).with_columns_renamed(old_names, new_names) 
                        for p in past_covs
                    ]
                    past_covs = [p.stack(p_ma) for p, p_ma in zip(past_covs, past_ma_covs)]
        
        past_dict[fam] = [p.astype(np.float32) for p in past_covs]

        # Extract future covariates
        future_covs = TimeSeries.from_group_dataframe(
            df=df,
            time_col="date",
            value_cols=future_cols,
            group_cols="store_nbr",
        )
        future_covs = [f.with_static_covariates(None) for f in future_covs]
        future_covs = covs_pipe.fit_transform(future_covs)
        
        # Add moving average features for future covariates if specified
        if future_ma_cols is not None and len(future_ma_cols) > 0:
            for size in future_window_sizes:
                ma_filter = MovingAverageFilter(window=size)
                future_ma_cols_filtered = [col for col in future_ma_cols if col not in excluded_features]
                if future_ma_cols_filtered:
                    old_names = [f"rolling_mean_{size}_{col}" for col in future_ma_cols_filtered]
                    new_names = [f"{col}_ma{size}" for col in future_ma_cols_filtered]
                    future_ma_covs = [
                        ma_filter.filter(f[future_ma_cols_filtered]).with_columns_renamed(old_names, new_names) 
                        for f in future_covs
                    ]
                    future_covs = [f.stack(f_ma) for f, f_ma in zip(future_covs, future_ma_covs)]
        
        future_dict[fam] = [f.astype(np.float32) for f in future_covs]
            
    return past_dict, future_dict

################################################################################
# EXTRACT FEATURES AND TIME SERIES DATA
################################################################################
# List of static covariates excluding 'store_nbr'; 'store_nbr' is automatically extracted using 'group_cols'
static_cols = ["city", "state", "type", "cluster"]

# Extract target time series
target_dict, pipe_dict, id_dict = get_target_series(static_cols)

# Define covariate columns
# Past covariates
past_cols = ["transactions"]

# Future covariates - focus on most important predictors
future_cols = [
    "oil", "onpromotion",
    "day", "month", "year", "day_of_week", 
    "sin_day_of_week", "cos_day_of_week", 
    "sin_month", "cos_month",
    "sin_day_of_year", "cos_day_of_year",
    "work_day",
]

# Add available holiday features
future_cols.extend(available_holidays)

# Add payday features
future_cols.extend(["is_payday", "is_day_after_payday", "is_payday_week"])

# Filter out low variance features
future_cols = [col for col in future_cols if col not in excluded_features]

# Additional past and future covariates from computing the moving averages
past_ma_cols = ["transactions"]  # Added transactions to moving averages
future_ma_cols = ["oil", "onpromotion"]

# Expanded window sizes for more comprehensive patterns
past_window_sizes = [7, 14, 28]
future_window_sizes = [7, 14, 28]

# Extract covariate time series
past_dict, future_dict = get_covariates(
    past_cols, 
    future_cols, 
    past_ma_cols, 
    future_ma_cols,
    past_window_sizes,
    future_window_sizes
)

# Display list of all covariates
if "AUTOMOTIVE" in past_dict:
    cprint(
        "List of all covariates:",
        "Static covariates:",
        static_cols + ["store_nbr"],
        "",
        "Past covariates:",
        past_dict["AUTOMOTIVE"][0].components.tolist(),
        "",
        "Future covariates:",
        future_dict["AUTOMOTIVE"][0].components.tolist(),
    )

################################################################################
# MODEL TRAINING CONFIGURATION
################################################################################
TRAINER_CONFIG = {
    # The time series data previously extracted
    "target_dict": target_dict,
    "pipe_dict": pipe_dict,
    "id_dict": id_dict,
    "past_dict": past_dict,
    "future_dict": future_dict,
    
    # Cross-validation settings
    "forecast_horizon": 16,  # The length of the validation set
    "folds": 1,  # Standard train-validation split (reduced from 3 to 1 for stability)
    
    # The number of previous days to check for zero sales; if all are zero, generate zero forecasts
    "zero_fc_window": 21,
    
    # Specify the covariates in a list to include in the model
    # Set to None to not use any, and set to 'keep_all' to include everything
    "static_covs": "keep_all",
    "past_covs": "keep_all",
    "future_covs": "keep_all",
}

################################################################################
# IMPROVED TRAINER CLASS DEFINITION
################################################################################
class Trainer:
    def __init__(
        self,
        target_dict,
        pipe_dict,
        id_dict,
        past_dict,
        future_dict,
        forecast_horizon,
        folds,
        zero_fc_window,
        static_covs=None,
        past_covs=None,
        future_covs=None,
    ):
        self.target_dict = target_dict.copy()
        self.pipe_dict = pipe_dict.copy()
        self.id_dict = id_dict.copy()
        self.past_dict = past_dict.copy()
        self.future_dict = future_dict.copy()
        self.forecast_horizon = forecast_horizon
        self.folds = folds
        self.zero_fc_window = zero_fc_window
        self.static_covs = static_covs
        self.past_covs = past_covs
        self.future_covs = future_covs
        
        self.setup()
    
    def setup(self):
        for fam in tqdm_notebook(self.target_dict.keys(), desc="Setting up"):
            if self.static_covs != "keep_all":
                if self.static_covs is not None:
                    target = self.target_dict[fam]
                    keep_static = [col for col in target[0].static_covariates.columns if col.startswith(tuple(self.static_covs))]
                    static_covs_df = [t.static_covariates[keep_static] for t in target]
                    self.target_dict[fam] = [t.with_static_covariates(d) for t, d in zip(target, static_covs_df)]
                else:
                    self.target_dict[fam] = [t.with_static_covariates(None) for t in self.target_dict[fam]]
            
            if self.past_covs != "keep_all":
                if self.past_covs is not None:
                    self.past_dict[fam] = [p[self.past_covs] for p in self.past_dict[fam]]
                else:
                    self.past_dict[fam] = None
                
            if self.future_covs != "keep_all":
                if self.future_covs is not None:
                    self.future_dict[fam] = [p[self.future_covs] for p in self.future_dict[fam]]
                else:
                    self.future_dict[fam] = None
    
    def clip(self, array):
        return np.clip(array, a_min=0., a_max=None)
    
    def train_valid_split(self, target, length):
        train = [t[:-length] for t in target]
        valid_end_idx = -length + self.forecast_horizon
        if valid_end_idx >= 0:
            valid_end_idx = None
        valid = [t[-length:valid_end_idx] for t in target]
        
        return train, valid
    
    def get_models(self, model_names, model_configs):
        models = {
            "lr": LinearRegressionModel,
            "lgbm": LightGBMModel,
            "cat": CatBoostModel,
            "xgb": XGBModel,
        }
        assert isinstance(model_names, list) and isinstance(model_configs, list),\
        "Both the model names and model configurations must be specified in lists."
        assert all(name in models for name in model_names),\
        f"Model names '{model_names}' not recognized."
        assert len(model_names) == len(model_configs),\
        "The number of model names and the number of model configurations do not match."
        
        if "xgb" in model_names:
            xgb_idx = np.where(np.array(model_names)=="xgb")[0]
            for idx in xgb_idx:
                model_configs[idx] = {"tree_method": "hist", **model_configs[idx]}
        
        return [models[name](**model_configs[j]) for j, name in enumerate(model_names)]
    
    def generate_forecasts(self, models, train, pipe, past_covs, future_covs, drop_before):
        """
        Generate forecasts using the trained models.
        
        Parameters:
        -----------
        models : list
            List of models to use for forecasting
        train : list
            List of training time series
        pipe : Pipeline
            Transformation pipeline for the target time series
        past_covs : list or None
            List of past covariates time series
        future_covs : list or None
            List of future covariates time series
        drop_before : str or None
            Date before which to drop data
            
        Returns:
        --------
        pred_list : list
            List of predictions for each model
        ens_pred : list
            Ensemble prediction (average of all models)
        """
        if drop_before is not None:
            date = pd.Timestamp(drop_before) - pd.Timedelta(days=1)
            train = [t.drop_before(date) for t in train]
            if past_covs is not None:
                past_covs = [p.drop_before(date) for p in past_covs]
            if future_covs is not None:
                future_covs = [f.drop_before(date) for f in future_covs]
        
        inputs = {
            "series": train,
            "past_covariates": past_covs,
            "future_covariates": future_covs,
        }
        
        # Create zero prediction template for items with no sales
        zero_pred = pd.DataFrame({
            "date": pd.date_range(train[0].end_time(), periods=self.forecast_horizon+1)[1:],
            "sales": np.zeros(self.forecast_horizon),
        })
        zero_pred = TimeSeries.from_dataframe(
            df=zero_pred,
            time_col="date",
            value_cols="sales",
        )
        
        pred_list = []
        ens_pred = [0 for _ in range(len(train))]
        
        for m in models:
            try:
                m.fit(**inputs)
            except Exception as e:
                print(f"Warning: Error during model fitting: {e}")
                continue

            try:
                pred = m.predict(n=self.forecast_horizon, **inputs)
                pred = pipe.inverse_transform(pred)

                # For time series with only zeros in recent history, predict zeros
                for j in range(len(train)):
                    if train[j][-self.zero_fc_window:].values().sum() == 0:
                        pred[j] = zero_pred
                
                # Clip predictions to ensure non-negative values
                pred = [p.map(self.clip) for p in pred]
                pred_list.append(pred)
                
                # Calculate ensemble prediction as average
                for j in range(len(ens_pred)):
                    ens_pred[j] += pred[j] / len(models)
            except Exception as e:
                print(f"Warning: Error during prediction: {e}")
                continue

        # If no models successfully generated predictions, return empty lists
        if not pred_list:
            print("Warning: No models generated predictions successfully.")
            return [], ens_pred

        return pred_list, ens_pred
    
    def metric(self, valid, pred):
        """Calculate Root Mean Squared Logarithmic Error (RMSLE)"""
        result = rmsle(valid, pred)
        if isinstance(result, list):
            return np.mean(result)
        return result
    
    def validate(self, model_names, model_configs, drop_before=None):
        longest_len = len(max(self.target_dict.keys(), key=len))
        
        model_metrics_history = []
        ens_metric_history = []
        
        for fam in tqdm_notebook(self.target_dict, desc="Performing validation"):
            target = self.target_dict[fam]
            pipe = self.pipe_dict[fam]
            past_covs = self.past_dict[fam]
            future_covs = self.future_dict[fam]
            
            model_metrics = []
            ens_metric = 0
            
            for j in range(self.folds):    
                length = (self.folds - j) * self.forecast_horizon
                train, valid = self.train_valid_split(target, length)
                valid = pipe.inverse_transform(valid)

                models = self.get_models(model_names, model_configs)
                pred_list, ens_pred = self.generate_forecasts(
                    models, train, pipe, past_covs, future_covs, drop_before
                )
                
                if pred_list:  # Check if any predictions were successfully generated
                    metric_list = [self.metric(valid, pred) / self.folds for pred in pred_list]
                    model_metrics.append(metric_list)
                    
                    if len(models) > 1:
                        ens_metric_fold = self.metric(valid, ens_pred) / self.folds
                        ens_metric += ens_metric_fold
                
            if model_metrics:  # Check if any metrics were calculated
                model_metrics = np.sum(model_metrics, axis=0)
                model_metrics_history.append(model_metrics)
                ens_metric_history.append(ens_metric)
                
                print(
                    fam,
                    " " * (longest_len - len(fam)),
                    " | ",
                    " - ".join([f"{model}: {metric:.5f}" for model, metric in zip(model_names, model_metrics)]),
                    f" - ens: {ens_metric:.5f}" if len(models) > 1 else "",
                    sep="",
                )
            
        if model_metrics_history:  # Check if any metrics were collected
            cprint(
                "Average RMSLE | "
                + " - ".join([f"{model}: {metric:.5f}" 
                              for model, metric in zip(model_names, np.mean(model_metrics_history, axis=0))])
                + (f" - ens: {np.mean(ens_metric_history):.5f}" if len(models) > 1 else ""),
            )
        else:
            print("No valid metrics could be calculated. Check model configurations.")
        
    def ensemble_predict(self, model_names, model_configs, drop_before=None):
        """Generate predictions using ensemble of models"""
        forecasts = []
        for fam in tqdm_notebook(self.target_dict.keys(), desc="Generating forecasts"):
            target = self.target_dict[fam]
            pipe = self.pipe_dict[fam]
            target_id = self.id_dict[fam]
            past_covs = self.past_dict[fam]
            future_covs = self.future_dict[fam]
            
            models = self.get_models(model_names, model_configs)
            
            # Use full dataset for final prediction
            _, ens_pred = self.generate_forecasts(models, target, pipe, past_covs, future_covs, drop_before)
            
            if not any(isinstance(p, TimeSeries) for p in ens_pred):
                print(f"Warning: Failed to generate predictions for {fam}")
                continue
                
            # Convert to dataframe and add identifiers
            ens_pred = [p.to_dataframe().assign(**i) for p, i in zip(ens_pred, target_id)]
            ens_pred = pd.concat(ens_pred, axis=0)
            forecasts.append(ens_pred)
            
        forecasts = pd.concat(forecasts, axis=0)
        forecasts = forecasts.rename_axis(None, axis=1).reset_index(names="date")
        
        return forecasts

################################################################################
# MODEL TRAINING AND EVALUATION
################################################################################
# Initialize model trainer
trainer = Trainer(**TRAINER_CONFIG)

# Base model configuration with improved regularization
BASE_CONFIG = {
    "random_state": 42,  # Consistent random state
    
    # The number of lag values of the target series
    "lags": 63,
    
    # The number of lag values of the past covariates
    "lags_past_covariates": list(range(-1, -30, -1)) if TRAINER_CONFIG["past_covs"] is not None else None,
    
    # The number of (past, future-1) lag values of the future covariates
    "lags_future_covariates": (14, 1) if TRAINER_CONFIG["future_covs"] is not None else None,
    
    # The number of days ahead that the model is forecasting given today's input data
    "output_chunk_length": 1,
}

# Validate linear regression model
# Note: Using only parameters supported by LinearRegressionModel in darts
LR_CONFIG = {
    **BASE_CONFIG,
    "fit_intercept": True,
}

print("\nValidating Linear Regression Model")
trainer.validate(["lr"], [LR_CONFIG], drop_before="2015-01-01")

# LightGBM configuration with stronger regularization to prevent overfitting
# Removed early_stopping_rounds to prevent errors
LGBM_BASE_CONFIG = {
    **BASE_CONFIG,
    
    # Core parameters
    "n_estimators": 200,
    "learning_rate": 0.03,  # Decreased to reduce overfitting
    "max_depth": 8,  # Reduced from 12 to prevent overfitting
    "num_leaves": 31,
    
    # Regularization parameters
    "min_child_samples": 20,  # Minimum samples per leaf
    "min_child_weight": 1e-3,  # Minimum sum of instance weight in a leaf
    "subsample": 0.7,  # Reduced from 0.8 for better regularization
    "colsample_bytree": 0.7,  # Reduced from 0.8 for better regularization
    "reg_alpha": 0.03,  # Increased L1 regularization
    "reg_lambda": 0.3,  # Increased L2 regularization
    
    # Removed early_stopping_rounds to avoid errors
    "verbose": -1,
}

# XGBoost configuration with strong regularization
# Removed early_stopping_rounds to prevent errors
XGB_CONFIG = {
    **BASE_CONFIG,
    "n_estimators": 300,
    "learning_rate": 0.03,
    "max_depth": 8,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    "reg_alpha": 0.05,
    "reg_lambda": 0.5,
    "gamma": 0.01,  # Minimum loss reduction for further partition
    "min_child_weight": 3,
    "tree_method": "hist",
    # Removed early_stopping_rounds to avoid errors
    "verbose": 0,
}

# Experiment with different lag configurations
LGBM_CONFIG1 = LGBM_BASE_CONFIG.copy()  # Keep full lag structure

LGBM_CONFIG2 = LGBM_BASE_CONFIG.copy()
LGBM_CONFIG2["lags"] = list(range(-1, -8, -1))  # One week of lags

LGBM_CONFIG3 = LGBM_BASE_CONFIG.copy()
LGBM_CONFIG3["lags"] = list(range(-365, -372, -1))  # Same week last year

# Add a configuration with very recent lags
LGBM_CONFIG5 = LGBM_BASE_CONFIG.copy()
LGBM_CONFIG5["lags"] = [-1, -2, -3]  # Very recent lags

# Add seasonal weekly pattern
LGBM_CONFIG6 = LGBM_BASE_CONFIG.copy()
LGBM_CONFIG6["lags"] = [-7, -14, -21, -28]  # Weekly patterns

# CatBoost configuration with regularization
CAT_CONFIG = {
    **BASE_CONFIG,
    "iterations": 200,
    "learning_rate": 0.03,
    "depth": 8,  # Reduced from 12
    "l2_leaf_reg": 5.0,  # Increased from 3.0
    "random_strength": 0.2,  # Increased from 0.1
    "bagging_temperature": 0.8,
    "grow_policy": "SymmetricTree",
    "verbose": False,
}

# Build a more robust ensemble with diversified models
# Using fewer models for better stability
ENS_MODELS = ["lr", "lgbm", "lgbm", "cat"]
ENS_CONFIGS = [
    LR_CONFIG,          # Linear model for baseline
    LGBM_CONFIG1,       # Full lag structure
    LGBM_CONFIG6,       # Weekly patterns
    CAT_CONFIG,         # CatBoost model
]

# Validate ensemble models
print("\nValidating Ensemble Models")
trainer.validate(
    model_names=ENS_MODELS, 
    model_configs=ENS_CONFIGS,
    drop_before="2015-01-01",
)

################################################################################
# GENERATE PREDICTIONS AND CREATE SUBMISSION FILE
################################################################################
# Generate predictions using the ensemble model
print("\nGenerating final predictions...")
final_predictions = trainer.ensemble_predict(
    model_names=ENS_MODELS,
    model_configs=ENS_CONFIGS,
    drop_before="2015-01-01",
)

def prepare_submission(predictions, test_df):
    """
    Prepare the final submission file.
    
    Parameters:
    -----------
    predictions : pd.DataFrame
        DataFrame containing predictions from the ensemble model
    test_df : pd.DataFrame
        The original test DataFrame with 'id' values
        
    Returns:
    --------
    pd.DataFrame
        DataFrame formatted for submission with 'id' and 'sales' columns
    """
    # Process store_nbr column - remove prefix added during preprocessing
    predictions.store_nbr = predictions.store_nbr.str.replace("store_nbr_", "", regex=True).astype(int)
     
    # Match predictions with corresponding 'id' from test set
    submission = test_df.merge(
        predictions, on=["date", "store_nbr", "family"], how="left"
    )[["id", "sales"]]
    
    return submission

# Prepare and save submission
print("\nPreparing submission file...")
submission = prepare_submission(final_predictions, test)

# Check if there are any missing values in the submission
missing_values = submission.isna().sum().sum()
if missing_values > 0:
    print(f"Warning: Submission contains {missing_values} missing values. Filling with zeros.")
    submission = submission.fillna(0)

# Save the submission file
submission_path = "submission.csv"
submission.to_csv(submission_path, index=False)
print(f"\nSubmission file saved to {submission_path}")

Basic information of data
Number of family types      : 33
Number of stores            : 54
Number of store-family pairs: 1782
Number of target series     : 1782

Number of unique train dates: 1684
Train date range            : 1688 days from 2013-01-01 to 2017-08-15
Number of unique test dates : 16
Test date range             : 16 days from 2017-08-16 to 2017-08-31
Missing gaps in time series
List incl. unique counts of data points: [1684]
Missing dates                          : ['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25']
Missing oil dates
Number of missing oil dates: 486
Number of weekends missing : 486
Total number of weekends   : 486
Missing transaction records
Correct number of records: 91152

Breakdown...
Current number of records: 83488
Number of zero sales     : 7546
Number of missing records: 118
List of locale names for each holiday
Locale names for national holidays:
['Ecuador']

Locale names for regional holidays:
['Cotopaxi', 'Imbabura', 'Santa Elena', 'Santo

Extracting target series:   0%|          | 0/33 [00:00<?, ?it/s]

Extracting covariates:   0%|          | 0/33 [00:00<?, ?it/s]

List of all covariates:
Static covariates:
['city', 'state', 'type', 'cluster', 'store_nbr']

Past covariates:
['transactions', 'transactions_ma7', 'transactions_ma14', 'transactions_ma28']

Future covariates:
['oil', 'onpromotion', 'day', 'month', 'year', 'day_of_week', 'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month', 'sin_day_of_year', 'cos_day_of_year', 'nat_terremoto', 'nat_navidad', 'is_payday', 'is_day_after_payday', 'is_payday_week', 'oil_ma7', 'onpromotion_ma7', 'oil_ma14', 'onpromotion_ma14', 'oil_ma28', 'onpromotion_ma28']


Setting up:   0%|          | 0/33 [00:00<?, ?it/s]

...

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_

SEAFOOD                    | lr: 0.44949 - lgbm: 0.45798 - lgbm: 0.45319 - cat: 0.46065 - ens: 0.44692
Average RMSLE | lr: 0.36255 - lgbm: 0.33700 - lgbm: 0.34278 - cat: 0.33983 - ens: 0.33220

Generating final predictions...


Generating forecasts:   0%|          | 0/33 [00:00<?, ?it/s]

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
`predict()` was called with `n > output_


Preparing submission file...

Submission file saved to submission.csv
